In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ./drive/MyDrive/Gaze_locking

/content/drive/.shortcut-targets-by-id/1cNr3ZUNb8Sfe_VVxQeMcQFyojgSJ3xjc/Gaze_locking


In [ ]:
# !unzip columbia_gaze_data_set.zip

In [ ]:
# import pandas as pd
# import os
# import re

# rootdir = './Columbia Gaze Data Set'
# data = []
# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         if 'jpg' in file.split('.'):
#           filedata = file.split('_')
#           temp = []
#           temp.append(filedata[0])
#           # print(filedata)
#           temp.append(int(re.findall(r'-?\d+',filedata[2])[0]))
#           temp.append(int(re.findall(r'-?\d+',filedata[3])[0]))
#           temp.append(int(re.findall(r'-?\d+',filedata[4])[0]))
#           temp.append(os.path.join(subdir, file))
#           if temp[2]==0 and temp[1]+temp[3]==0:
#               temp.append(True)
#           else:
#               temp.append(False)

#           # print(temp)
#           # break
#           data.append(temp)
#           # print(os.path.join(subdir, file))

# df = pd.DataFrame(data, columns=['person','pose','vertical','horizontal','path','is_gaze'])

# df.to_csv('gaze_data.csv')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def detect_eyepatch(face_image):
  gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')
  faces = face_cascade.detectMultiScale(gray, 1.3, 4)
  x,y,w,h = faces[0]
  roi_gray = gray[y:y+h, x:x+w]
  roi_color = face_image[y:y+h, x:x+w]
  eyes = eye_cascade.detectMultiScale(roi_gray)
  eye_strip = []
  for (ex,ey,ew,eh) in eyes[:2]:
    mid_x = int(ex+ ew/2)
    mid_y = int(ey + eh/2)
    eye_strip.append(roi_gray[ey:ey+eh,ex:ex+ew])

  #   cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
  # cv2_imshow(face_image)
  return eye_strip

# a = detect_eyepatch(cv2.imread('./Columbia Gaze Data Set/0001/0001_2m_-15P_-10V_5H.jpg'))
  # cv2.rectangle(roi_color,(mid_x-24,mid_y-18),(mid_x+24,mid_y+18),(0,255,255),2)
# cv2_imshow(face_img)


In [ ]:
import numpy as np
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow

def getCorners(eye_patch):
  corner1 = cv2.goodFeaturesToTrack(eye_patch[:,:12], 1, 0.01, 10)
  corner2 = cv2.goodFeaturesToTrack(eye_patch[:,36:], 1, 0.01, 10)
  corner1 = np.int0(corner1)
  corner2 = np.int0(corner2)
  corner1 = [corner1[0][0][0],corner1[0][0][1]]
  corner2 = [corner2[0][0][0]+36,corner2[0][0][1]]
  return [corner1,corner2]


In [ ]:
def transform(eye_patch):
  transformation_matrix=cv2.getPerspectiveTransform(np.float32([[0,0],[eye_patch.shape[1],0],[0,eye_patch.shape[0]],[eye_patch.shape[1],eye_patch.shape[0]]]),np.float32([
          [0,0],
          [48,0],
          [0,36],
          [48,36]
      ]))
  rectified_eye_patch = cv2.warpPerspective(eye_patch,transformation_matrix,(48,36),flags=cv2.INTER_LINEAR)
  return rectified_eye_patch


In [ ]:
import matplotlib.patches as patches
import numpy as np
import math

def elliptical_mask(eye_patch,corners):
  g_ell_center = ((corners[1][0]+corners[0][0])/2, (corners[0][1]+corners[1][1])/2)
  g_ell_width = math.dist(corners[0],corners[1])
  g_ell_height = g_ell_width/1.8

  angle = np.rad2deg(np.arctan((corners[1][1]-corners[0][1])/(corners[1][0]-corners[0][0])))

  g_ellipse = patches.Ellipse(g_ell_center, g_ell_width, g_ell_height, angle=angle, fill=False, edgecolor='green', linewidth=2)
  cos_angle = np.cos(np.radians(180.-angle))
  sin_angle = np.sin(np.radians(180.-angle))
  x = np.array([[i for i in range(48)] for i in range(36)])
  y = np.array([[j for i in range(48)]for j in range(36)])
  xc = x - g_ell_center[0]
  yc = y - g_ell_center[1]
  xct = xc * cos_angle - yc * sin_angle
  yct = xc * sin_angle + yc * cos_angle
  rad_cc = (xct**2/(g_ell_width/2.)**2) + (yct**2/(g_ell_height/2.)**2)
  eye_patch = np.where(rad_cc>1. , 0 , eye_patch)
  return eye_patch


Train-test split

In [ ]:
import random
import pandas as pd
import pickle as pkl
df = pd.read_csv('./gaze_data.csv')
train_test_ratio = 0.9
indexes = [ind for ind in df.index]
random.shuffle(indexes)
n = len(indexes)
train = indexes[:int(train_test_ratio*n)]
test = indexes[int(train_test_ratio*n):]

with open('./feature_matrix.pkl','rb') as file:
  feature_matrix = pkl.load(file)
with open('./labels.pkl','rb') as file:
  labels = pkl.load(file)

with open('./hlabels.pkl','rb') as file:
  hlabels = pkl.load(file)

Training the data

In [ ]:
# import os
# import pandas as pd
# import random
# import pickle
# feature_matrix_t=[]
# img_folder=os.listdir("./Columbia Gaze Data Set")
# # print(img_folder)
# labels = []
# df = pd.read_csv('./gaze_data.csv')

# n = len(indexes)

# print(len(train))
# horizontal_labels=[]
# for ind in train:
#       # print(file)
#       # if file.endswith('.jpg'):


#     face_img = cv2.imread(df['path'][ind])
#     try:
#       eye_strip = detect_eyepatch(face_img)
#       transformed_eye_strip = list(map(transform,eye_strip))
#       corners = list(map(getCorners,transformed_eye_strip))
#       masked_left_eye=elliptical_mask(transformed_eye_strip[0],corners[0])
#       masked_right_eye=elliptical_mask(transformed_eye_strip[1],corners[1])
#       final_eye_strip=np.hstack((masked_left_eye,masked_right_eye))
#       #cv2_imshow(final_eye_strip)
#       final_eye_strip=np.matrix.flatten(final_eye_strip)
#       feature_matrix_t.append(final_eye_strip)
#       labels.append(int(df['is_gaze'][ind]))
#       horizontal_labels.append(int((int(df['horizontal'][ind])+15)/5))
#       print(int((int(df['horizontal'][ind])+15)/5))
#     except:
#       continue
#     #print(final_eye_strip.shape)
# with open('feature_matrix.pkl', 'wb') as file:

#     # A new file will be created
#     pickle.dump(feature_matrix, file)

# with open('labels.pkl', 'wb') as file:
#   pickle.dump(labels, file)

# with open('horizontal_labels.pkl', 'wb') as file:
#   pickle.dump(horizontal_labels, file)



In [ ]:
from sklearn.decomposition import PCA
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import normalize
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
import pickle as pkl

def training_models(feature_matrix,labels,hlabels,da_model):
  pca = PCA(n_components=200)
  transformed_data = pca.fit_transform(normalize(feature_matrix,axis=1))
  pred = []
  # transformed_data = pca.fit_transform(feature_matrix,axis=1)
  with open('pca_model.pkl', 'wb') as file:
    pkl.dump(pca, file)
  if da_model=='only_svm':
    svm=SVC(kernel='rbf')
    svm.fit(transformed_data,labels)
    with open(f'svm_model_{da_model}.pkl', 'wb') as file:
        # A new file will be created
        pkl.dump(svm, file)
    return

  if da_model=='lda':
    #LDA with assumption equal covariance matrices
    clf = LinearDiscriminantAnalysis(n_components=6)
    transformed_data_da=clf.fit_transform(transformed_data,hlabels)
    # svm=LinearSVC()
    svm=SVC(kernel='rbf')
    svm.fit(transformed_data_da,labels)
    pred = svm.predict(transformed_data_da)

  if da_model=='qda':
    clf = QuadraticDiscriminantAnalysis()
    transformed_data_qda=clf.fit(transformed_data,hlabels)
    # svm=LinearSVC()
    svm=SVC(kernel='rbf')
    svm.fit(transformed_data_qda.decision_function(transformed_data),labels)
    pred = svm.predict(transformed_data_qda.decision_function(transformed_data))

  if da_model=='mda':
      n_classes = 7

      # Create a pipeline with MDA
      gmm = GaussianMixture(n_components=n_classes)
      clf = LinearDiscriminantAnalysis()
      # clf = make_pipeline(gmm, lda)

      # Fit the pipeline to the data
      # gmm.fit(transformed_data)
      gmm_data = gmm.fit_predict(transformed_data)
      transformed_data_da=clf.fit_transform(np.reshape(gmm_data,(gmm_data.shape[0],1)),hlabels)
      with open(f'gmm.pkl','wb') as file:
        pkl.dump(gmm, file)
      svm=SVC(kernel='rbf',)
      svm.fit(transformed_data_da,labels)
      pred = svm.predict(transformed_data_da)

  with open(f'{da_model}_model.pkl','wb') as file:
    pkl.dump(clf, file)


  with open(f'svm_model_{da_model}.pkl', 'wb') as file:

    # A new file will be created
    pkl.dump(svm, file)


  return confusion_matrix(labels,pred)


# training_models(feature_matrix,labels,hlabels,'lda')
# training_models(feature_matrix,labels,hlabels,'qda')
# training_models(feature_matrix,labels,hlabels,'mda')
conf = training_models(feature_matrix,labels,hlabels,'only_svm')
print(conf)

Testing

In [ ]:
def test_pipeline(probe_img):
  try:
    predicted_label=[]
    p_eye_strip = detect_eyepatch(probe_img)
    p_transformed_eye_strip = list(map(transform,p_eye_strip))
    p_corners = list(map(getCorners,p_transformed_eye_strip))

    p_masked_left_eye=elliptical_mask(p_transformed_eye_strip[0],p_corners[0])
    p_masked_right_eye=elliptical_mask(p_transformed_eye_strip[1],p_corners[1])
    p_final_eye_strip=np.hstack((p_masked_left_eye,p_masked_right_eye))

    with open('pca_model.pkl', 'rb') as file:

        # Call load method to deserialze
        pca = pkl.load(file)

    p_pca=pca.transform(normalize(p_final_eye_strip.reshape(1,3456),axis=1))

    with open('svm_model_only_svm.pkl','rb') as file:
        svm = pkl.load(file)
    predicted_label.append(svm.predict(p_pca))
    # p_pca=pca.transform(p_final_eye_strip.reshape(1,3456),axis=1)
    # if da_model=='lda':
    with open('lda_model.pkl','rb') as file:
      clf = pkl.load(file)
    p_da=clf.transform(p_pca)
    with open('svm_model_lda.pkl','rb') as file:
      svm = pkl.load(file)
    # predicted_label=svm.predict(p_da)
    predicted_label.append(svm.predict(p_da))


    # if da_model=='qda':
    with open('qda_model.pkl','rb') as file:
      clf = pkl.load(file)
    p_da=clf.decision_function(p_pca)
    with open('svm_model_qda.pkl','rb') as file:
      svm = pkl.load(file)
    # predicted_label=svm.predict(p_da)
    predicted_label.append(svm.predict(p_da))


    # if da_model=='mda':
    with open('gmm.pkl','rb') as file:
      gmm = pkl.load(file)
    with open('mda_model.pkl','rb') as file:
      clf = pkl.load(file)
    gmm_data = gmm.predict(p_pca)
    p_da=clf.transform(np.reshape(gmm_data,(gmm_data.shape[0],1)))
    with open('svm_model_mda.pkl','rb') as file:
      svm = pkl.load(file)
    # predicted_label=svm.predict(p_da)
    predicted_label.append(svm.predict(p_da))


    return predicted_label
  except:
    return [-1,-1,-1,-1]
# img = cv2.imread(df['path'][0])
# a = test_pipeline(img,'mda')
# print(a)

tp = [0,0,0,0]
fp = [0,0,0,0]
fn = [0,0,0,0]
tn = [0,0,0,0]
for ind in test:
  df = pd.read_csv('./gaze_data.csv')
  label = int(df['is_gaze'][ind])
  img = cv2.imread(df['path'][ind])
  pred = []
  pred=test_pipeline(img)
  # pred.append(test_pipeline(img,'qda'))
  # pred.append(test_pipeline(img,'mda'))

  for i in range(4):
    if pred[i]==-1:
      continue
    elif pred[i]==label:
      if label==0:
        tn[i]+=1
      else:
        tp[i]+=1
    else:
      if pred[i]==0:
        fn[i]+=1
      else:
        fp[i]+=1


NameError: ignored

In [ ]:
# Initialize lists to store metrics
precision = []
recall = []
accuracy = []
specificity = []
f1_score = []
mcc = []

# Calculate metrics for each set of values
for i in range(len(tp)):
    # Accuracy
    acc = (tp[i] + tn[i]) / (tp[i] + tn[i] + fp[i] + fn[i])
    accuracy.append(acc)

    # Precision
    prec = tp[i] / (tp[i] + fp[i]) if (tp[i] + fp[i]) > 0 else 0
    precision.append(prec)

    # Recall
    rec = tp[i] / (tp[i] + fn[i]) if (tp[i] + fn[i]) > 0 else 0
    recall.append(rec)

    # Specificity
    spec = tn[i] / (tn[i] + fp[i]) if (tn[i] + fp[i]) > 0 else 0
    specificity.append(spec)

    # F1 Score
    f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0
    f1_score.append(f1)

    # Matthews Correlation Coefficient (MCC)
    numerator = (tp[i] * tn[i]) - (fp[i] * fn[i])
    denominator = ((tp[i] + fp[i]) * (tp[i] + fn[i]) * (tn[i] + fp[i]) * (tn[i] + fn[i])) ** 0.5
    mcc_val = numerator / denominator if denominator != 0 else 0
    mcc.append(mcc_val)

# Print the calculated metrics
for i in range(len(tp)):
    print(f"Set {i + 1} Metrics:")
    print(f"Accuracy: {accuracy[i]:.4f}")
    print(f"Precision: {precision[i]:.4f}")
    print(f"Recall: {recall[i]:.4f}")
    print(f"Specificity: {specificity[i]:.4f}")
    print(f"F1 Score: {f1_score[i]:.4f}")
    print(f"MCC: {mcc[i]:.4f}")
    print()



Set 1 Metrics:
Accuracy: 0.9473
Precision: 0.2000
Recall: 0.0800
Specificity: 0.9858
F1 Score: 0.1143
MCC: 0.1027

Set 2 Metrics:
Accuracy: 0.9456
Precision: 0.2667
Recall: 0.1600
Specificity: 0.9805
F1 Score: 0.2000
MCC: 0.1797

Set 3 Metrics:
Accuracy: 0.9422
Precision: 0.2000
Recall: 0.1200
Specificity: 0.9787
F1 Score: 0.1500
MCC: 0.1263

Set 4 Metrics:
Accuracy: 0.9592
Precision: 0.5333
Recall: 0.3200
Specificity: 0.9876
F1 Score: 0.4000
MCC: 0.3936

